In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm


In [ ]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.token_to_id = {word: idx for idx, word in enumerate(vocab)}
        self.id_to_token = {idx: word for idx, word in enumerate(vocab)}

    def tokenize(self, text):
        return [word for word in text.split() if word in self.vocab]

    def encode(self, text):
        return [self.token_to_id[word] for word in self.tokenize(text)]

    def decode(self, tokens):
        return ' '.join([self.id_to_token[token] for token in tokens])

vocab = ["[PAD]", "[CLS]", "[SEP]", "hello", "how", "are", "you"]
tokenizer = SimpleTokenizer(vocab)

#Cơ chế self_attention:Cơ chế cốt lõi của mô hình.
##Các biến chính:
* embed_size: size của vector embedding
* heads: số head_attention --> chạy song song
* head_dim: chiều mỗi head(embed_size//head)
* values,keys,queries(V,K,Q):
* fc_out
# Forward pass(def forward)
* Reshape
* Tính attention
* thay




In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out



In [ ]:
class BERT(nn.Module):
    def __init__(
        self,
        embed_size,
        num_layers,
        heads,
        vocab_size,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(BERT, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size, heads, dropout=dropout, forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.word_embedding(x) + self.position_embedding(positions)

        for layer in self.layers:
            out = layer(out, out, out, mask)

        out = self.fc(out)
        return out


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERT(embed_size=256, num_layers=6, heads=8, vocab_size=len(vocab), forward_expansion=4, dropout=0, max_length=100).to(device)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tokens = self.tokenizer.encode(text)
        tokens = tokens + [0] * (100 - len(tokens))  # Padding to fixed length
        return torch.tensor(tokens)

texts = ["hello how are you", "how are you", "hello are you"]
dataset = TextDataset(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

def train(model, dataloader, optimizer, criterion, device, epochs=3):
    model.train()
    for epoch in range(epochs):
        for batch in tqdm(dataloader):
            batch = batch.to(device)
            mask = (batch != 0).unsqueeze(1).unsqueeze(2)
            outputs = model(batch, mask)
            loss = criterion(outputs.view(-1, outputs.shape[-1]), batch.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

train(model, dataloader, optimizer, criterion, device)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.eval